# Kazalo
- [O vodilih](../../../../code/01.Introduce)
- [Nastavitev](../../../../code/01.Introduce)
- [Neomejeno generiranje](../../../../code/01.Introduce)
- [Govorjenje za Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Izbira](../../../../code/01.Introduce)
- [Veriga misli](../../../../code/01.Introduce)
- [Generiranje JSON](../../../../code/01.Introduce)
- [Generiranje HTML](../../../../code/01.Introduce)


# O Guidance

Guidance je preverjena odprtokodna Python knjižnica za nadzor izhodov katerega koli jezikovnega modela (LM). Z enim API klicem lahko v Pythonu izrazite natančne programatske omejitve, ki jih mora model upoštevati, in ustvarite strukturiran izhod v JSON, Python, HTML, SQL ali katero koli strukturo, ki jo zahteva primer uporabe.

Guidance se razlikuje od običajnih tehnik pozivanja. Omejitve uveljavlja tako, da model usmerja token za tokenom v sloju sklepanja, kar omogoča bolj kakovostne izhode ter zmanjšuje stroške in zakasnitve za kar 30–50 % pri uporabi za visoko strukturirane scenarije.

Če želite izvedeti več o Guidance, obiščite [javno skladišče na GitHubu](https://github.com/guidance-ai/guidance) ali si oglejte [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) na Microsoft Build.


# Namestitev
1. Namestite Guidance z `pip install guidance --pre`
2. Ustvarite mini končno točko Phi 3.5 v Azure tako, da obiščete https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml in kliknete gumb "Deploy"
3. Shranite API ključ vaše končne točke v okoljsko spremenljivko z imenom `AZURE_PHI3_KEY` in URL v okoljsko spremenljivko z imenom `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Neposredno generiranje
Besedilo je mogoče generirati brez omejitev z uporabo funkcije `gen()`. To je enako kot uporaba modela brez Guidance.

## Oblikovanje klepeta
Tako kot mnogi modeli za klepet, Phi-3 pričakuje sporočila med uporabnikom in asistentom v določenem formatu. Guidance podpira Phi-3 predlogo za klepet in bo za vas upravljal oblikovanje klepeta. Za ustvarjanje izmenjav v klepetu postavite vsak del pogovora v blok `with user()` ali `with assistant()`. Blok `with system()` se lahko uporabi za nastavitev sistemskega sporočila.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Prihranek žetonov
V zelo strukturiranih scenarijih lahko Guidance preskoči žetone in ustvari le potrebne žetone, kar izboljša zmogljivost, poveča učinkovitost in zmanjša stroške API-ja. Ustvarjeni žetoni so v tem zvezku prikazani z označenim ozadjem. Prisilni žetoni so prikazani brez označevanja in stanejo enako kot vhodni žetoni, katerih stroški so ocenjeni na eno tretjino stroškov izhodnih žetonov.

*Opomba:* Prvi primer z neomejenim generiranjem ni mogel prisiliti nobenih žetonov, ker nismo podali nobenih omejitev.


# Govorimo o Phi 3
S pomočjo Guidance lahko preprosto vstavite besedilo v odgovore modela. To je lahko koristno, če želite usmeriti izhod modela v določeno smer.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Omejevanje z regex
V prejšnjem primeru je Phi 3 podal dodatne razlage po odgovoru na vprašanje z `Canberra`. Da bi omejili izhod modela na točno eno besedo, lahko uporabimo regex.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Z regexom je ustvarjena samo beseda `Canberra`.


# Izbira med več možnostmi
Ko so nekatere možne izbire znane, lahko uporabite funkcijo `select()`, da model izbere iz seznama možnosti.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Z `select()` je bil ustvarjen le žeton `Can`. Ker je `Canberra` edina možnost, ki lahko dokonča odgovor, so bili preostali žetoni prisiljeni.


# Veriga razmišljanja
Veriga razmišljanja je tehnika, ki lahko izboljša kakovost izhoda modela, tako da ga spodbuja k postopnemu reševanju problema. Običajno je za dosego končnega odgovora potrebnih več korakov pozivanja. Najprej modelu naročite, naj razmišlja korak za korakom. Nato model ponovno pozovete, da poda končni odgovor. Pri standardnih API-jih za sklepanje v klepetu to zahteva 2 klica API-ja, pri čemer se modelova generirana "veriga razmišljanja" zaračuna dvakrat – enkrat kot izhodni žetoni, ko jo model ustvari, in nato še enkrat kot vhodni žetoni za drugi klic. Z Guidance pa se celoten večstopenjski proces obravnava in zaračuna kot del enega samega klica API-ja, kar zmanjša stroške in zakasnitve.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generiranje JSON-a
Guidance se lahko uporablja za zagotavljanje generiranja JSON-a, ki je skladen z JSON shemo ali pydantic modelom, kot je prikazano v shemi uporabniškega profila tukaj.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Generiranje HTML-ja

Guidance se lahko uporablja za generiranje kode in upoštevanje sintaktičnih zahtev programskega jezika. V tem razdelku bomo ustvarili majhen Guidance program za pisanje zelo preprostih HTML spletnih strani.

Spletno stran bomo razdelili na manjše odseke, pri čemer bo vsak imel svojo Guidance funkcijo. Te funkcije bomo nato združili v naši končni funkciji za ustvarjanje HTML spletne strani.
Na koncu bomo to funkcijo izvedli z modelom, ki podpira Guidance, v Azure AI.

*Opomba:* To ne bo popolnoma funkcionalen generator HTML-ja; cilj je pokazati, kako lahko ustvarite strukturiran izhod za svoje specifične potrebe.

Začnemo z uvozom potrebnega iz Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML spletne strani so zelo strukturirane, in te dele strani bomo 'prisilili' z uporabo Guidance.  
Ko izrecno zahtevamo besedilo od modela, moramo zagotoviti, da ne vključuje ničesar, kar bi lahko bilo oznaka - torej moramo izključiti znaka '<' in '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Nato lahko uporabimo to funkcijo za generiranje besedila znotraj poljubne oznake HTML:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Zdaj ustvarimo glavo strani.  
Kot del tega moramo ustvariti naslov strani:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Telo HTML strani bo napolnjeno z naslovi in odstavki.  
Lahko definiramo funkcijo za vsakega:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Zdaj funkcija za definiranje telesa samega HTML-ja.  
Ta uporablja `select()` z `recurse=True` za generiranje več naslovov in odstavkov:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Nato pridemo do funkcije, ki ustvari celotno HTML stran.  
Dodamo začetno oznako HTML, nato ustvarimo glavo, nato telo, in na koncu dodamo zaključni HTML oznako:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Ponujamo uporabniku prijazen vmesnik, ki nam omogoča:
- Nastavitev temperature generiranja
- Zajem generirane strani iz objekta Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Nato lahko zapišemo izhod v datoteko:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

In [oglejte si rezultat](../../../../code/01.Introduce/sample_page.html).



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitne nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
